In [1]:
import json
from tqdm import tqdm
import time

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client=Elasticsearch("http://localhost:9200")

In [6]:
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7b3ed9690580>: Failed to establish a new connection: [Errno 111] Connection refused))

In [12]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


In [14]:
index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [16]:
for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████| 948/948 [00:26<00:00, 36.08it/s]


In [18]:
query="How do i run Kafka"

In [21]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

In [23]:
response=es_client.search(index=index_name,body=search_query)

In [24]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'DcXJJZABarbO4fWCrMVG',
  '_score': 30.447786,
  '_source': {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
   'section': 'Workshop 1 - dlthub',
   'question': 'How do I install the necessary dependencies to run the code?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '9sXJJZABarbO4fWCqcTI',
  '_score': 26.374605,
  '_source': {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
   'section': 'Module 6: streaming with kafka',
   'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-